In [1]:
import pandas as pd 

In [2]:
df_validation=pd.read_csv('../data/sales_train_validation.csv',dtype='int32')
df_sells=pd.read_csv('../data/sell_prices.csv')
df_calendar=pd.read_csv('../data/calendar.csv')

In [4]:
all_cals=df_validation.columns.to_list()+df_calendar.columns.to_list()+df_sells.columns.to_list()

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_validation=pd.melt(
    df_validation,
    id_vars=df_validation.columns[:6],
    value_vars=df_validation.columns[6:],
    var_name='day',
    value_name='sales'
)

In [6]:
df_validation['day']=df_validation['day'].str[2:]

In [4]:
print(df_validation.shape)
df_validation.head()

(58327370, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,day,sales
0,14370,1437,3,1,0,0,1,0
1,14380,1438,3,1,0,0,1,0
2,14390,1439,3,1,0,0,1,0
3,14400,1440,3,1,0,0,1,0
4,14410,1441,3,1,0,0,1,0


In [8]:
df_validation.to_csv('../data/sales_train_validation.csv',index=False,header=True)

# now we merge the 3 datasets 

In [6]:
df_sells.head()

,store_id,item_id,wm_yr_wk,sell_price
0,0,1437,11325,9.58
1,0,1437,11326,9.58
2,0,1437,11327,8.26
3,0,1437,11328,8.26
4,0,1437,11329,8.26


In [8]:
df_calendar.head()


,wm_yr_wk,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,weekday_cos,weekday_sin,day
0,11101,1,2011,1,19,2,3,1,0,0,0,-0.222521,-0.974928,29
1,11101,1,2011,2,19,2,3,1,0,0,0,0.623490,-0.781831,30
2,11101,1,2011,3,19,2,3,1,0,0,0,1.000000,0.000000,31
3,11101,2,2011,4,19,2,3,1,1,1,0,0.623490,0.781831,1
4,11101,2,2011,5,19,2,3,1,1,0,1,-0.222521,0.974928,2


In [3]:
# we join calendar and sells on wm_yr_wk

df_mrg=pd.merge(
    df_validation,
    df_calendar,
    how='inner',
    left_on='day',
    right_on='d'
)#should have 100% join rate 
print(df_mrg.shape)

(58327370, 22)


In [4]:
df_mrg.to_csv('first_merge.csv',header=True,index=False)

In [2]:
df_mrg=pd.read_csv('first_merge.csv',nrows=1)
cols={}
for col in df_mrg.columns : 
    if col=='weekday_cos' or col=='weekday_sin' : 
        continue 
    cols[col]='int32'
print(cols)

{'id': 'int32', 'item_id': 'int32', 'dept_id': 'int32', 'cat_id': 'int32', 'store_id': 'int32', 'state_id': 'int32', 'day_x': 'int32', 'sales': 'int32', 'wm_yr_wk': 'int32', 'month': 'int32', 'year': 'int32', 'd': 'int32', 'event_name_1': 'int32', 'event_type_1': 'int32', 'event_name_2': 'int32', 'event_type_2': 'int32', 'snap_CA': 'int32', 'snap_TX': 'int32', 'snap_WI': 'int32', 'day_y': 'int32'}


In [3]:
df_mrg=pd.read_csv('first_merge.csv',dtype=cols)

In [4]:
df_sells=pd.read_csv('../data/sell_prices.csv',dtype={'store_id':'int32','item_id':'int32','wm_yr_wk':'int32'})

In [5]:
df_mrg=pd.merge(
    df_mrg,
    df_sells,
    how='left',
    on=['store_id','item_id','wm_yr_wk']
)
print(df_mrg.shape)

(58327370, 23)


In [6]:
df_mrg.to_csv('final.csv',header=True,index=False)